# GPU Ocean Tutorial - Deterministic Trajectory Forecast

This tutorial will show how to set up and run a deterministic drift trajectory forecast using real-world data.

A breif overview of the most important classes and methods in this respect is given. The tutorial then covers selecting an area and loading boundary- and initial conditions, setting up a simulator and observation-instance, adding drifters and running a simulation. It also covers certain basic configurations and how to save, access and plot trajectory results. Saving and accessing data about the state of the underlying ocean model is not included.  

## Overview of classes and modules

<img src="cropped_figure-1[1].png" width=600 height=600 />

The figure above summarizes key elements when simulating a (deterministic) drift trajectory. Classes and methods are in blue, green represent data-files. 

We now move on to an example. We start by importing the relevant modules and setting the 

In [2]:
#Lets have matplotlib "inline"
%matplotlib inline

import os
import sys

#Import packages we need
import numpy as np
from netCDF4 import Dataset
import datetime
from IPython.display import display

#For plotting
import matplotlib
from matplotlib import pyplot as plt

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../../')))

GPU Ocean-modules:

In [3]:
from SWESimulators import CDKLM16, Common, IPythonMagic, NetCDFInitialization
from SWESimulators import GPUDrifterCollection, Observation
from SWESimulators import DataAssimilationUtils as dautils

from demos.realisticSimulations import norkyst_plotting

In [4]:
%cuda_context_handler gpu_ctx

## Selecting area and loading initial- and boundary conditions

As initial and boundary conditions to the simulation, we use data from the Norkyst800 model:

In [5]:
#source_url = 'https://thredds.met.no/thredds/dodsC/fou-hi/norkyst800m-1h/NorKyst-800m_ZDEPTHS_his.an.2019071600.nc'
#locally:
source_url = '/home/johanna/gpu-ocean/gpu_ocean/demos/MPI_SIR/netcdf_cache/NorKyst-800m_ZDEPTHS_his.an.2019071600.nc'

A local file path or URL to a NetCDF-file, or lists of either may be given as argument. 

Before loading the data, we need to specify what domain to model: 

  <code> NetCDFInitialization.getInitialConditions(source_url, x0, x1, y0, y1) </code>
    
loads the data relevant for the domain limited by $X[x_0], X[x_1], Y[y_0], Y[y_1]$, where $X$, $Y$ are horizontal position arrays from the given netCDF-file. That is, $x_0, x_1, y_0, y_1$ are indicies into position arrays, which in this case span a $800$m resolution grid. 

Certain areas within the Norkyst-800 model domain are pre-defined. We can access these with

   <code> NetCDFInitialization.getInitialConditionsNorkystCases(source_url, casename)</code>

We choose Lofoten:

In [6]:
casename = 'lofoten'
data_args = NetCDFInitialization.getInitialConditionsNorKystCases(source_url,casename, download_data=False)

## Creating simulator and forecast

Stripping data_args of unneccessary information with  <code> NetCDFInitialization.removeMetadata</code> it can then be passed to a simulator object along with some additional arguments:

In [7]:
sim_args = {
    "gpu_ctx": gpu_ctx,
    "dt": 0.0,
    #"desingularization_eps": 1.0 Why this exactly?
     }

sim = CDKLM16.CDKLM16(**sim_args, **NetCDFInitialization.removeMetadata(data_args))

This will give inaccurate angle along the border!
This will give inaccurate coriolis along the border!


When nothing else is specified, a second order runge kutta method is used.

We create an observation-object to store the forecast, using parameters from the simulation :

In [8]:
observation_args = {'observation_type': dautils.ObservationType.UnderlyingFlow,
                'nx': sim.nx, 'ny': sim.ny,
                'domain_size_x': sim.nx*sim.dx,
                'domain_size_y': sim.ny*sim.dy,
                'land_mask': sim.getLandMask()
               }

forecast = Observation.Observation(**observation_args)

For objects drifting at sea, UnderlyingFlow is the correct ObservationType.

## Creating drifters

Now we create some drifters. The drifter collection will be attached to the simulation later. This means the simulation of drift trajectories can start at any point during an ongoing ocean model simulation. 

In [9]:
num_drifters = 3
drifters = GPUDrifterCollection.GPUDrifterCollection(gpu_ctx, num_drifters, 
                                                 boundaryConditions = sim.boundary_conditions,
                                                 domain_size_x = forecast.domain_size_x,
                                                 domain_size_y = forecast.domain_size_y,
                                                 gpu_stream = sim.gpu_stream)


We set the drifter positions with <code> .setDrifterPositions</code>:

In [10]:
init_positions = np.array([[39189.93, 38952.23],  #[[x1, y1],
                           [38762.76, 38783.02],  # [x2, y2],
                           [39128.16, 38939.48]]) # [x3, y3]]
drifters.setDrifterPositions(init_positions)

## Running the simulation

<code> Simulator.step(dt) </code> evolves the simulation dt forward in time. Looping over this, we can simulate the ocean using a stepsize dt of our choice and for the number of hours we want(limited by the available boundary-data), attaching drifters and starting the trajectory forecast after a chosen number of hours.

<code> Observation.add_observation_from_sim </code> saves the current positions of drifters to the observation object in each time step. 

Here, we simulate the ocean for 24 hours, adding drifters after the first 5.

In [11]:
start_drifters_hours = 5
total_hours = 24
dt = 5*60 # Stepsize: 5min (in seconds)
timesteps_per_hour = int(60*60 / dt)

#Create progress bar
progress = Common.ProgressPrinter(5)
pp = display(progress.getPrintString(0), display_id=True)

#Run simulation
for hour in range(total_hours):
    
    if hour == start_drifters_hours:                  #Attach drifters 
        sim.attachDrifters(drifters)
        forecast.add_observation_from_sim(sim)
    
    for n in range(timesteps_per_hour):    
        t = sim.step(dt)                              #make one step forward in time
        if hour >= start_drifters_hours:               #Add positions to observation only if drifters have been attached
            forecast.add_observation_from_sim(sim) 
    
    #Update progress bar
    pp.update(progress.getPrintString(hour/(total_hours-1)))

'0% [##############################] 100%. Total: 1m 31s, elapsed: 1m 31s, remaining: 0s'

## Saving trajectories to file

The observation class provides functions <code>.to_pickle</code> and <code>.read_pickle</code> to easily write and read trajectory info to and from file. 

In [12]:
filename = 'tutorial_forecast_'+str(start_drifters_hours)+'_to_'+str(total_hours)+'.pickle'

#save data
forecast.to_pickle(filename)

#Retrieve data
forecast_from_file = Observation.Observation(**observation_args) # Create new observation for the trajectories to be stored
forecast_from_file.read_pickle(filename)                         # Load pickled observations into new observation object

## Accessing and plotting the results